In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
from PIL import Image
def crop(image_path, coords, saved_location):
    """
    @param image_path: The path to the image to edit
    @param coords: A tuple of x/y coordinates (x1, y1, x2, y2)
    @param saved_location: Path to save the cropped image
    """
    image_obj = Image.open(image_path)
    cropped_image = image_obj.crop(coords)
    cropped_image = cropped_image.resize( (64, 64), Image.LANCZOS)
    cropped_image.save(saved_location)

In [ ]:
# crops the images

import os
from PIL import Image

PATH = '../crowdai_train/'
UNCROPPED = 'uncropped/'
CROPPED = 'cropped/'

trainingFolders = os.listdir(PATH + UNCROPPED)
count = 0
for i in range(2):
    folder = trainingFolders[i]
    images = os.listdir(PATH + UNCROPPED + '/' + folder)
    for image in images:
        wantedDimen = 224
        im = Image.open(PATH + UNCROPPED + '/' + folder + '/' + image)
        width, height = im.size
        
        wMargin = width - wantedDimen
        hMargin = height - wantedDimen
        
        if wMargin < 0 or hMargin < 0:
            continue
        
        # check if folder exists and make it if it doesn't
        directory = PATH + CROPPED + folder
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        crop( PATH + UNCROPPED + folder + '/' + image,
            (wMargin // 2,
            hMargin // 2,
            wMargin // 2 + wantedDimen,
            hMargin // 2 + wantedDimen),
            PATH + CROPPED + folder + '/' + image)
        

In [ ]:
import os
import matplotlib.image as mimg
import numpy as np

# Get train and test sets
CROPPED_IMAGES_FOLDER = '../crowdai_train/cropped/'

images_train_x = []
images_train_y = []

folder_index = 0
for imageFolder in os.listdir(CROPPED_IMAGES_FOLDER):
    for image in os.listdir(CROPPED_IMAGES_FOLDER + imageFolder):
        im = mimg.imread(CROPPED_IMAGES_FOLDER + imageFolder + '/' + image)
        images_train_x.append(im)
        images_train_y.append(folder_index)
    folder_index += 1
        
images_train_x = np.array(images_train_x)
images_train_y = np.array(images_train_y)

images_train_x[0]
size_x = images_train_x.shape[0]
train_index = int(size_x * 0.8)
test_index = int(size_x * 0.2)

x_train, x_test = images_train_x[:train_index,:], images_train_x[test_index:,:]
y_train, y_test = images_train_y[:train_index], images_train_y[test_index:]

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt

batch_size = 64
num_classes = 2
epochs = 10

# input image dimensions
img_x, img_y = 64, 64


# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
# x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 3)
# x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 3)
input_shape = (img_x, img_y, 3)

# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
plt.plot(range(1, 11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [13]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt


batch_size = 128
num_classes = 2
epochs = 10

# input image dimensions
img_x, img_y = 28, 28

# load the MNIST data set, which already splits into train and test sets for us
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [18]:
import numpy as np
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  